## Task 3: Proximity Analysis (using Population Centers)
Regions to analyze: Lima and Loreto

Methodology:
1. From Population Centers, compute the centroid of each locality (or use the provided geometry centroids).
2. For each centroid, calculate the number of operational hospitals within a 10 km buffer.
3. Identify (per region):
- The population center with the fewest hospitals nearby (isolation).
- The population center with the most hospitals nearby (concentration).
4. Plot with folium:
- The selected population center (centroid).
- The 10 km buffer (circle).
- All hospitals inside the buffer.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

### Paso 1: From Population Centers, compute the centroid of each locality (or use the provided geometry centroids).

In [2]:
# Cargamos los centros poblados (INEI)
ccpp = gpd.read_file("../Output/CCPP_0/CCPP_IGN100K.shp")
ccpp

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
0,1,PANDISHARI,INEI,2502010002,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.06462,-10.37129,PANDISHARI,POINT (-74.06462 -10.37129)
1,2,CHICOSA,INEI,2502010003,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.06153,-10.37852,CHICOSA,POINT (-74.06153 -10.37852)
2,3,RAYA,IGN,2502010004,OTROS,RAYMONDI,ATALAYA,UCAYALI,2350,Centro Poblado Menor,-72.94118,-10.33043,RAYA,POINT (-72.94118 -10.33043)
3,4,PENSILVANIA,INEI,2502010005,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.05988,-10.40401,PENSILVANIA,POINT (-74.05988 -10.40401)
4,5,PONTE VEDRA,INEI,2502010006,CASERÍO,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.03788,-10.41809,PONTE VEDRA,POINT (-74.03787 -10.41809)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136582,136584,IPAN,INEI,None,None,ZORRITOS,CONTRALMIRANTE VILLAR,TUMBES,None,None,-81.02462,-81.02462,None,POINT (-81.02462 -4.09728)
136583,136585,URBAN,INEI,None,None,ZORRITOS,CONTRALMIRANTE VILLAR,TUMBES,None,None,-80.84055,-80.84055,None,POINT (-80.84055 -4.06801)
136584,136586,PAJONAL MAJONTONI,IGN,None,None,RAYMONDI,ATALAYA,UCAYALI,2049,Centro Poblado Menor,-74.35804,-74.35804,PAJONAL MAJONTONI,POINT (-74.35804 -10.7274)
136585,136587,AGUA BLANCA,INEI,2501020043,CASERÍO,CAMPOVERDE,CORONEL PORTILLO,UCAYALI,1953,Centro Poblado Menor,-74.78089,-74.78089,AGUA BLANCA,POINT (-74.78089 -8.60511)


In [3]:
# La geometría son puntos. Usamos la geometría ya definida en el shapefile
ccpp = gpd.read_file("../Output/CCPP_0/CCPP_IGN100K.shp")

# Nos aseguras que todo quede en el CRS correcto (EPSG:4326).
if ccpp.crs is None:
    ccpp = ccpp.set_crs("EPSG:4326")
else:
    ccpp = ccpp.to_crs("EPSG:4326")

In [4]:
# Filtramos para Lima y Loreto
ccpp_lima = ccpp[ccpp["DEP"] == "LIMA"].copy()
ccpp_loreto = ccpp[ccpp["DEP"] == "LORETO"].copy()

print("Centros poblados en Lima:", len(ccpp_lima))
print("Centros poblados en Loreto:", len(ccpp_loreto))

Centros poblados en Lima: 7615
Centros poblados en Loreto: 3447


### Paso 2: For each centroid, calculate the number of operational hospitals within a 10 km buffer.

In [5]:
# Cargamos la data de los IPRESS y filtramos los que: pertenecen al MINSA, estén en funcionamiento, con coordenadas válidas y que solo sean hospitales
cv_data = pd.read_csv("../Output/IPRESS.csv", encoding="latin1")
hosp = cv_data[cv_data["Condición"] == "EN FUNCIONAMIENTO"].copy()
hosp = hosp.dropna(subset=["NORTE", "ESTE"])
hosp = hosp[hosp["Clasificación"].isin([
    "HOSPITALES O CLINICAS DE ATENCION GENERAL",
    "HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA"
])]
hosp = hosp[~hosp["Institución"].str.contains("PRIVADO", na=False)]

In [6]:
# Convertimos a GeoDataFrame
gdf_hosp = gpd.GeoDataFrame(
    hosp,
    geometry=gpd.points_from_xy(hosp["NORTE"], hosp["ESTE"]),  # ESTE=X, NORTE=Y
    crs="EPSG:4326"
)


In [7]:
#Nos quedamos solo con Lima y Loreto

hosp_lima = gdf_hosp[gdf_hosp["Departamento"] == "LIMA"].copy()
hosp_loreto = gdf_hosp[gdf_hosp["Departamento"] == "LORETO"].copy()

print("Hospitales en Lima:", len(hosp_lima))
print("Hospitales en Loreto:", len(hosp_loreto))

Hospitales en Lima: 42
Hospitales en Loreto: 6


In [8]:
# Reproyectamos hospitales y centros poblados a CRS métrico (UTM zona 18S)
gdf_hosp_utm = gdf_hosp.to_crs(epsg=32718)
ccpp_lima_utm = ccpp_lima.to_crs(epsg=32718)
ccpp_loreto_utm = ccpp_loreto.to_crs(epsg=32718)

In [9]:
# Crear buffers en metros (EPSG:32718)
ccpp_lima_buf = ccpp_lima_utm.copy()
ccpp_lima_buf["geometry"] = ccpp_lima_buf.buffer(10000)

ccpp_loreto_buf = ccpp_loreto_utm.copy()
ccpp_loreto_buf["geometry"] = ccpp_loreto_buf.buffer(10000)

# Spatial join: hospitales que caen dentro del buffer
join_lima = gpd.sjoin(gdf_hosp_utm, ccpp_lima_buf, how="inner", predicate="within")
join_loreto = gpd.sjoin(gdf_hosp_utm, ccpp_loreto_buf, how="inner", predicate="within")

print("Join Lima:", join_lima.shape)
print("Join Loreto:", join_loreto.shape)


Join Lima: (2676, 48)
Join Loreto: (283, 48)


### Paso 3: Identify (per region): 
(1) The population center with the fewest hospitals nearby (isolation).

(2) The population center with the most hospitals nearby (concentration).

In [10]:
# 1) Elegimos un identificador único de hospital (mejor que el nombre, pues puede haber centros poblados con el mismo nombre)
hosp_id_col = "Código Único" if "Código Único" in join_lima.columns else "Nombre del establecimiento"

# 2) Llave única del centro poblado (evita homónimos)
cp_key = ["DEP", "PROV", "DIST", "NOM_POBLAD"]

# --- PARA LIMA ---
# Quitamos hospitales sin ID y deduplicamos por (CP, hospital)
jl_lima_clean = (
    join_lima
    .dropna(subset=[hosp_id_col])
    .drop_duplicates(subset=cp_key + [hosp_id_col])
)

# Conteo de hospitales únicos por centro poblado (clave completa)
conteo_lima = (
    jl_lima_clean
    .groupby(cp_key)[hosp_id_col]
    .nunique()
    .reset_index(name="n_hosp")
    .sort_values("n_hosp", ascending=False)
)

# Si quieres incluir CPs con 0 hospitales (opcional):
cp_lima_full = ccpp_lima[cp_key].drop_duplicates()
conteo_lima = (
    cp_lima_full.merge(conteo_lima, on=cp_key, how="left")
    .assign(n_hosp=lambda df: df["n_hosp"].fillna(0).astype(int))
)

# --- PARA LORETO ---
jl_loreto_clean = (
    join_loreto
    .dropna(subset=[hosp_id_col])
    .drop_duplicates(subset=cp_key + [hosp_id_col])
)

conteo_loreto = (
    jl_loreto_clean
    .groupby(cp_key)[hosp_id_col]
    .nunique()
    .reset_index(name="n_hosp")
    .sort_values("n_hosp", ascending=False)
)

cp_loreto_full = ccpp_loreto[cp_key].drop_duplicates()
conteo_loreto = (
    cp_loreto_full.merge(conteo_loreto, on=cp_key, how="left")
    .assign(n_hosp=lambda df: df["n_hosp"].fillna(0).astype(int))
)

# Selección de extremos (si hay empates, toma el primero)
iso_lima = conteo_lima.loc[conteo_lima["n_hosp"].idxmin()]
conc_lima = conteo_lima.loc[conteo_lima["n_hosp"].idxmax()]
iso_loreto = conteo_loreto.loc[conteo_loreto["n_hosp"].idxmin()]
conc_loreto = conteo_loreto.loc[conteo_loreto["n_hosp"].idxmax()]

print("Aislamiento Lima:", iso_lima.to_dict())
print("Concentración Lima:", conc_lima.to_dict())
print("Aislamiento Loreto:", iso_loreto.to_dict())
print("Concentración Loreto:", conc_loreto.to_dict())



Aislamiento Lima: {'DEP': 'LIMA', 'PROV': 'CONCEPCION', 'DIST': 'SAN JOSE DE QUERO', 'NOM_POBLAD': 'RIEGOPAMPA', 'n_hosp': 0}
Concentración Lima: {'DEP': 'LIMA', 'PROV': 'LIMA', 'DIST': 'LIMA', 'NOM_POBLAD': 'LIMA', 'n_hosp': 26}
Aislamiento Loreto: {'DEP': 'LORETO', 'PROV': 'UCAYALI', 'DIST': 'CONTAMANA', 'NOM_POBLAD': 'PUEBLO NUEVO', 'n_hosp': 0}
Concentración Loreto: {'DEP': 'LORETO', 'PROV': 'MAYNAS', 'DIST': 'IQUITOS', 'NOM_POBLAD': 'IQUITOS', 'n_hosp': 3}


### Paso 4: Plot with folium:
- The selected population center (centroid).
- The 10 km buffer (circle).
- All hospitals inside the buffer.


In [11]:
import folium

def plot_all_regions_iso_conc(ccpp_lima, hosp_lima, iso_lima, conc_lima,
                              ccpp_loreto, hosp_loreto, iso_loreto, conc_loreto,
                              zoom_start=5):
    """
    Generaremos un mapa que contenga:
      - Todos los hospitales en gris
      - Centros poblados seleccionados (aislamiento y concentración) para Lima y Loreto
      - Buffers de 10 km
      - Hospitales dentro de cada buffer resaltados
    """

    # Creamos mapa centrado en Perú
    m = folium.Map(location=[-9.19, -75.015], zoom_start=zoom_start)

    # --- Función auxiliar ---
    def add_region(ccpp_region, gdf_hosp_region, iso_result, conc_result, color_iso, color_conc):
        iso_geom = ccpp_region.loc[ccpp_region["NOM_POBLAD"] == iso_result["NOM_POBLAD"], "geometry"].iloc[0]
        conc_geom = ccpp_region.loc[ccpp_region["NOM_POBLAD"] == conc_result["NOM_POBLAD"], "geometry"].iloc[0]

        # Buffers en EPSG:32718 → volver a EPSG:4326
        iso_buffer = gpd.GeoSeries([iso_geom], crs="EPSG:4326").to_crs(32718).buffer(10000).to_crs(4326).iloc[0]
        conc_buffer = gpd.GeoSeries([conc_geom], crs="EPSG:4326").to_crs(32718).buffer(10000).to_crs(4326).iloc[0]

        # Todos los hospitales de la región en gris
        for _, row in gdf_hosp_region.iterrows():
            folium.CircleMarker(
                [row.geometry.y, row.geometry.x],
                radius=2, color="gray", fill=True, fill_opacity=0.5
            ).add_to(m)

        # --- Aislamiento ---
        folium.Marker(
            [iso_geom.y, iso_geom.x],
            popup=f"Aislamiento: {iso_result['NOM_POBLAD']} ({iso_result['n_hosp']} hospitales)",
            icon=folium.Icon(color=color_iso)
        ).add_to(m)
        folium.GeoJson(iso_buffer, style_function=lambda x: {"color": color_iso, "fill": False}).add_to(m)

        for _, row in gdf_hosp_region[gdf_hosp_region.within(iso_buffer)].iterrows():
            folium.CircleMarker(
                [row.geometry.y, row.geometry.x],
                radius=4, color=color_iso, fill=True
            ).add_to(m)

        # --- Concentración ---
        folium.Marker(
            [conc_geom.y, conc_geom.x],
            popup=f"Concentración: {conc_result['NOM_POBLAD']} ({conc_result['n_hosp']} hospitales)",
            icon=folium.Icon(color=color_conc)
        ).add_to(m)
        folium.GeoJson(conc_buffer, style_function=lambda x: {"color": color_conc, "fill": False}).add_to(m)

        for _, row in gdf_hosp_region[gdf_hosp_region.within(conc_buffer)].iterrows():
            folium.CircleMarker(
                [row.geometry.y, row.geometry.x],
                radius=4, color=color_conc, fill=True
            ).add_to(m)

    # Agregar Lima (rojo/verde)
    add_region(ccpp_lima, hosp_lima, iso_lima, conc_lima, color_iso="red", color_conc="green")

    # Agregar Loreto (azul/puple para diferenciarlos)
    add_region(ccpp_loreto, hosp_loreto, iso_loreto, conc_loreto, color_iso="blue", color_conc="purple")

    return m


In [12]:
map_all = plot_all_regions_iso_conc(
    ccpp_lima, hosp_lima, iso_lima, conc_lima,
    ccpp_loreto, hosp_loreto, iso_loreto, conc_loreto,
    zoom_start=5
)
map_all


In [13]:
# Guardamos en HTML dentro de output
map_all.save("../Output/mapa_lima_loreto.html")